In [242]:
import json
import urllib
from bs4 import BeautifulSoup
from collections import OrderedDict
import numpy as np
import pandas as pd
import copy
# load rmagic
#%load_ext rpy2.ipython

In [249]:
# this cell is most likely useless

def extract_mapping(id_value, xml=idd_structure.html.body):
    """
    stats.oecd.org sends back results in the same order as the
    structure specified in XML body, so we need to use
    an ordered dictionary to preserve the ordering
    """
    output = OrderedDict()
    subtree = xml.find(id=id_value)
    for x in subtree.find_all('code'):
        output[x['value']] = x.find(attrs={'xml:lang':'en'}).text
    
    return output

# build function to extract mapping from data structure
idd_structure=BeautifulSoup(urllib.urlopen('http://stats.oecd.org/restsdmx/sdmx.ashx/GetDataStructure/IDD'))

"""
the 'keyfamily' tags in the structure contain the id's of all the dimension-data
<KeyFamily id="IDD" agencyID="OECD">
    <Name xml:lang="en">Income Distribution and Poverty</Name>
    <Name xml:lang="fr">Distribution des revenus et pauvreté</Name>
    <Components>
        <Dimension codelist="CL_IDD_LOCATION" conceptRef="LOCATION"/>
        <Dimension codelist="CL_IDD_MEASURE" conceptRef="MEASURE" isMeasureDimension="true"/>
        <Dimension codelist="CL_IDD_AGE" conceptRef="AGE"/>
        <Dimension codelist="CL_IDD_DEFINITION" conceptRef="DEFINITION"/>
        <Dimension codelist="CL_IDD_METHODO" conceptRef="METHODO"/>
        <TimeDimension codelist="CL_IDD_TIME" conceptRef="TIME"/>
        ...
    </Components>
...
</KeyFamily>

We can then search the dom for valid dimensions values via
dom.find_all(id=dimension['codelist'])
"""
dimension_maps = {}
for x in idd_structure.html.body.keyfamily.components.find_all('dimension'):
    dimension_maps[x['conceptref'].lower()]=extract_mapping(x['codelist'])

""" build API query:
The format is 

LOCATION.MEASURE.AGE.DEFINITION.METHODO

where each dimension is '+' separated

For the most part, we're interest all locations,
measures, ages, and methods, and just the 'CURRENT'
dimension...
age and definition only return one value...
"""
dimensions = ['location', 'measure', 'age', 'definition', 'methodo']
dimension_filter = []
for dimension in dimensions:
    dimension_map = dimension_maps[dimension]
    dimension_filter.append('+'.join(dimension_map.keys()))

dimension_filter = '.'.join(dimension_filter)

### Look at Income Disparity

In [304]:
idd_base_url = 'http://stats.oecd.org/sdmx-json/data/IDD'
dimension_filter = 'all' # grab all data
time_filter = 'all?startTime=2001&endTime=2014'
optional_filters = '&detail=dataonly'
data_query = idd_base_url + '/' + dimension_filter + '/' + time_filter + optional_filters
stats_oecd_idd = json.load(urllib.urlopen(data_query))

In [363]:
"""parse json data into DataFrame:

the json comes with a 'structure' section which explains how to
interpret the keys (e.g. 1:2:3:4:5) for the observations.

We first need to parse the 'structure' to figure out what the
dimension names (and order) is. Then, we need to unpack
observation values into floats (as opposed to arrays)
"""
metadata_stats_oecd_idd = stats_oecd_idd['structure']['dimensions']

col_names = []
for dim in metadata_stats_oecd_idd['series']:
    col_names.append(dim['id'].lower())

idd_dataframe = pd.DataFrame()

for key, value in stats_oecd_idd['dataSets'][0]['series'].items()[0:100]:
    print key
    data = pd.DataFrame(dict(zip(col_names, [[x] for x in key.split(":")])))
    
    observations = copy.copy(value['observations'])
    
    for time, measure in observations.items():
        observations[time] = measure[0]
    
    observations = pd.DataFrame(observations.items(), columns=['time_period','observation'])
    
    observations['location'] = data['location'].iloc[0]
    
    data = pd.merge(right=data,\
                    left=observations,\
                    on='location',\
                    how='outer')
    try:
        idd_dataframe = idd_dataframe.append(data, ignore_index=False)
    except AttributeError:
        pass
    

4:60:0:0:0
23:62:1:0:1
1:0:0:2:0
24:37:1:2:0
33:25:0:0:1
16:1:0:0:0
16:1:0:0:1
2:39:1:0:1
13:64:1:0:1
7:32:0:0:0
11:14:0:0:0
11:14:0:0:1
3:15:2:0:1
8:37:1:0:0
29:33:1:0:1
29:33:1:0:0
22:46:0:0:1
22:46:0:0:0
1:30:0:0:0
1:30:0:0:1
6:37:0:0:0
6:37:0:0:1
2:45:1:0:1
29:2:0:0:1
29:2:0:0:0
0:63:1:0:0
0:63:1:0:1
23:15:1:0:1
23:15:1:0:0
31:17:1:0:0
27:48:2:0:0
27:48:2:0:1
25:9:2:0:0
12:53:0:0:0
12:53:0:0:1
11:1:0:0:1
11:1:0:0:0
21:51:1:0:0
9:3:1:0:0
9:3:1:0:1
16:51:1:0:1
16:51:1:0:0
9:43:0:0:0
9:43:0:0:1
6:27:1:0:1
18:31:0:0:0
18:31:0:0:1
1:14:2:0:0
1:14:2:0:1
13:37:2:0:1
13:37:2:0:0
33:2:0:0:1
26:47:0:0:0
26:47:0:0:1
10:2:0:0:1
10:2:0:0:0
4:0:0:0:1
4:0:0:0:0
23:47:0:0:1
23:47:0:0:0
10:49:0:0:1
10:49:0:0:0
5:28:0:0:1
5:28:0:0:0
12:15:2:0:0
29:38:0:0:1
29:38:0:0:0
24:44:2:0:1
34:24:1:0:0
24:27:0:2:0
19:25:0:1:1
3:15:2:0:0
25:22:0:0:0
25:22:0:0:1
24:51:1:0:0
24:51:1:0:1
30:63:1:0:1
4:0:2:0:1
7:9:2:0:0
4:0:2:0:0
20:3:1:1:1
20:62:1:1:1
1:50:0:0:0
1:50:0:0:1
23:16:0:0:1
23:16:0:0:0
27:40:1:0:1
13:14

In [364]:
"""
Append meta data to OECD stats dataframe:

the returned JSON has a 'structure' object with 'dimensions'
child that contains all the information to translate series
identifiers x:y:z:w:a into something human parsable.

A series of merges should be anticipated. Hence, it makes
sense to store all the metadata in a dictionary where the
keys will be the merge indices.
"""
oecd_metadata = {}
for dim in metadata_stats_oecd_idd['series']:
    dim_dataframe = pd.DataFrame(dim['values'])
    dim_id = dim['id'].lower()
    dim_dataframe.columns = [dim_id + '_code', dim_id + '_name']
    dim_dataframe[dim_id] = dim_dataframe.index.astype(str)
    oecd_metadata[dim_id] = dim_dataframe

# make time metadata
time_metadata = metadata_stats_oecd_idd['observation'][0]
time_metadata_index = time_metadata['role'].lower()
time_metadata_df = pd.DataFrame(time_metadata['values'])
time_metadata_df.columns = ['year', time_metadata_index]
time_metadata_df[time_metadata_index] = time_metadata_df.index.astype(str)

oecd_metadata[time_metadata_index] = time_metadata_df

for key, df in oecd_metadata.items():
    idd_dataframe = pd.merge(left=idd_dataframe,\
                             right=df,\
                             how='left',\
                             on=key)

In [370]:
print idd_dataframe[idd_dataframe.measure_code == 'GINI'][\
                    idd_dataframe.definition_code=='CURRENT'][\
                    idd_dataframe.age_code=='TOT'][\
                    idd_dataframe.methodo_code=='METH2011'].sort('year')

    time_period  observation location age definition measure methodo  \
378          10        0.260        4   0          0       0       1   
379           1        0.269        4   0          0       0       1   
385           7        0.261        4   0          0       0       1   
387           8        0.261        4   0          0       0       1   
384           4        0.257        4   0          0       0       1   
382           2        0.260        4   0          0       0       1   
383           5        0.258        4   0          0       0       1   
381           3        0.258        4   0          0       0       1   
386           6        0.256        4   0          0       0       1   
380           0        0.252        4   0          0       0       1   

    definition_code     definition_name age_code          age_name  \
378         CURRENT  Current definition      TOT  Total population   
379         CURRENT  Current definition      TOT  Total population 

In [225]:
X = stats_oecd_idd['dataSets'][0]['series']
zip(dimensions, [[x] for x in X.keys()[0].split(':')])

#print X['0:0:0:0:1']
#print [x.keys()[0] for x in dimension_maps.values()] 
#idd_dataframe.sort(['location','age', 'definition', 'measure','time','methodology'])
#print dimension_maps['location'].keys() # CAN == 7
data_string = str(dimension_maps['location'].keys().index('CAN')) + ':' +\
str(dimension_maps['measure'].keys().index('GINI')) + ":" +\
str(dimension_maps['age'].keys().index('TOT')) + ":" +\
str(dimension_maps['definition'].keys().index('CURRENT')) + ":" +\
str(dimension_maps['methodo'].keys().index('METH2012'))
print data_string

print idd_dataframe.apply(lambda x: len(set(x.astype(int))), axis=0)
# 14 years
# 35 locations
# 3 ages
# 3 defintions
# 65 measures
# 2 methodologies
for key, values in dimension_maps.items():
    print key +":"+ str(len(values.keys()))

7:32:2:0:0
time              14
observation    12284
location          35
age                3
definition         3
measure           65
methodology        2
dtype: int64
methodo:2
definition:3
age:3
location:36
measure:70


In [322]:
idd_dataframe

In [12]:
# save IDD data to csv
idd_dataframe.to_csv('idd_dataframe.csv', index=False)

In [6]:
# this would be where I use Rmagic but it's failing
# so I'm gonna spin up an R notebook instead
%%R -i idd_dataframe -o x
cat("hello")
x <- typeof(x)


SyntaxError: invalid syntax (<ipython-input-6-04f7236fd6d0>, line 3)

In [11]:
idd_dataframe.to_csv?